In [16]:
# pip install openai

from openai import AzureOpenAI

ENDPOINT = "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1"
API_KEY = "f839c878-573c-4d2d-984d-0c70a8618775"

API_VERSION = "2024-02-01"
MODEL_NAME = "gpt-4o"

client = AzureOpenAI(
    azure_endpoint=ENDPOINT,
    api_key=API_KEY,
    api_version=API_VERSION,
)

In [24]:
import joblib
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

In [28]:
# Load the pre-trained model and preprocessor
model = load_model('average_score_model.h5', compile=False)
model.compile(optimizer='adam', loss='mse')

preprocessor = joblib.load('preprocessor.pkl')

In [29]:
def predict_average_score(preprocessed_input):
    """
    Predict the average score based on the preprocessed input.

    Args:
    preprocessed_input (numpy.array): The preprocessed input ready for model prediction.

    Returns:
    float: The predicted average score.
    """
    # The model expects a 2D array for input, ensure preprocessed_input is correctly shaped
    prediction = model.predict(preprocessed_input)
    return prediction[0][0]  # Extracting the score from the prediction result

def preprocess_user_input(user_attributes):
    """
    Transform raw user input to be suitable for model prediction.
    
    Args:
    user_attributes (dict): A dictionary containing user attributes with keys matching model feature names.

    Returns:
    numpy.array: The preprocessed data ready for model prediction.
    """
    # Convert the user attributes dictionary into a DataFrame
    user_df = pd.DataFrame([user_attributes])  # Wrapping in a list to handle single-row creation

    # Use the preprocessor to transform the data
    user_input_transformed = preprocessor.transform(user_df)
    
    return user_input_transformed

In [30]:
def generate_result(user_input):
    # Prepare the input for prediction
    preprocessed_input = preprocess_user_input(user_input)
    
    # Get the predicted average score from the model
    predicted_score = predict_average_score(preprocessed_input)

    MESSAGES = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"I need help creating a study plan. Based on my habits, my predicted average score is around {predicted_score}. I want to improve this score."},
        {
            "role": "assistant",
            "content": "Sure, I'd be happy to help! What subjects or topics do you need to include in your study plan?",
        },
        {"role": "user", "content": "The plan should cover mathematics over the next month."},
        {
            "role": "assistant",
            "content": "Do you have any specific goals or exams for these subjects during this period?",
        },
        {"role": "user", "content": "Yes, I'm preparing for midterms and need to cover calculus in mathematics."}
    ]



    completion = client.chat.completions.create(
        model=MODEL_NAME,
        # prompt=start_prompt,
        messages=MESSAGES
    )

    response_content = completion.choices[0].message.content
    return response_content

In [38]:
# Example usage:
user_attributes = {
    'Gender': 'Male',
    'EthnicGroup': 'Group B',
    'ParentEduc': "Bachelor's degree",
    'LunchType': 'Standard',
    'TestPrep': 'None',
    'ParentMaritalStatus': 'Married',
    'PracticeSport': 'Yes',
    'IsFirstChild': 'No',
    'WklyStudyHours': '10',
    'MathScore': 88, 
    'ReadingScore': 90,
    'WritingScore': 87
}

output = generate_result(user_attributes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [40]:
print(output)

Great, let's break down the study plan into weekly goals and daily tasks to ensure comprehensive coverage of the calculus material. Here's a sample one-month study plan:

### Week 1: Review Fundamentals
**Goal:** Ensure a strong foundation in basic calculus concepts.

**Day 1:**
- Review limits and continuity
- Practice problems on limits (30 minutes)

**Day 2:**
- Study the concept of derivatives
- Practice problems on finding derivatives (30 minutes)

**Day 3:**
- Review differentiation techniques (product rule, quotient rule, chain rule)
- Practice problems using different techniques (30 minutes)

**Day 4:**
- Study applications of derivatives (tangent lines, rates of change, etc.)
- Practice application problems (30 minutes)

**Day 5:**
- Review critical points and the first derivative test
- Practice problems on finding and classifying critical points (30 minutes)

**Day 6:**
- Study the second derivative test and concavity
- Practice problems on concavity and inflection points (3

In [13]:
# import pickle

# def predict_average_score(user_input):
#     # Load the model from disk (consider loading this once outside the function for efficiency)
#     with open('average_score_model.pkl', 'rb') as file:
#         model = pickle.load(file)
    
#     # Assuming user_input is appropriately formatted for the model
#     predicted_score = model.predict([user_input])[0]  # Assuming the model expects a list of inputs
#     return predicted_score

In [15]:
predict_model

array(['Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep',
       'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild',
       'WklyStudyHours', 'MathScore', 'ReadingScore', 'WritingScore'],
      dtype=object)